In [29]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

RANDOM_STATE = 42

from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Reshape
from keras.layers import Conv2D, Convolution3D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import pescador

%matplotlib inline
import matplotlib.pyplot as plt

import glob
import keras
import medleydb as mdb
from medleydb import utils
import numpy as np
import os
import pescador

In [38]:
def keras_generator(data_list, input_patch_size,
                    output_patch_size,
                    with_replacement=True,
                    batch_size=1024):
    streams = []
    for fpath in data_list:
        streams.append(
            pescador.Streamer(
                patch_generator, fpath,
                input_patch_size=input_patch_size,
                output_patch_size=output_patch_size
            )
        )

    stream_mux = pescador.Mux(
        streams, 3, with_replacement=with_replacement, lam=1000
    )

    buffered_streamer = pescador.BufferedStreamer(stream_mux, batch_size)
#     batch_generator = pescador.buffer_batch(stream_mux.generate(), batch_size)

    return buffered_streamer
#     for batch in buffered_streamer:
#         yield (batch['X'], batch['Y'])


def __grab_patch_output(f, t, n_f, n_t, y_data):
    return y_data[f: f + n_f, t: t + n_t][np.newaxis, :, :]


def __grab_patch_input(f, t, n_f, n_t, n_harms, x_data):
    return np.transpose(
        x_data[:, f: f + n_f, t: t + n_t], (1, 2, 0)
    )[np.newaxis, :, :, :]


def patch_generator(fpath, input_patch_size, output_patch_size):
    data = np.load(fpath, mmap_mode='r')

    n_harms, n_freqs, n_times = data['data_in'].shape
    n_f_in, n_t_in = input_patch_size
    n_f_out, n_t_out = output_patch_size

    f_shift = n_f_in - n_f_out
    t_shift = n_t_in - n_t_out
    
    while True:
        f = np.random.randint(0, n_freqs - n_f_in)
        t = np.random.randint(0, n_times - n_t_in)

        x = __grab_patch_input(
            f, t, n_f_in, n_t_in, n_harms, data['data_in']
        )
        y = __grab_patch_output(
            f + f_shift, t + t_shift, n_f_out, n_t_out, data['data_out']
        )
        yield (x, y)
#         yield dict(X=x, Y=y)


def stride_tf(fpath, input_patch_size, output_patch_size):
    data = np.load(fpath, mmap_mode='r')
    n_harms, n_freqs, n_times = data['data_in'].shape

    n_f_in, n_t_in = input_patch_size
    n_f_out, n_t_out = output_patch_size

    f_shift = n_f_in - n_f_out
    t_shift = n_t_in - n_t_out

    f_indices = np.arange(0, n_freqs - n_f, n_f)
    t_indices = np.arange(0, n_times - n_t, n_t)

    for f in f_indices:
        for t in t_indices:
            x = __grab_patch_input(
                f, t, n_f_in, n_t_in, n_harms, data['data_in']
            )
            y = __grab_patch_output(
                f + f_shift, t + t_shift, n_f_out, n_t_out, data['data_out']
            )
            yield dict(X=x, Y=y, t=t, f=f)


def get_file_paths(mtrack_list, data_path):
    file_paths = []
    for track_id in mtrack_list:
        file_paths.extend(
            glob.glob(os.path.join(data_path, "{}*.npz".format(track_id)))
        )
    return file_paths

In [31]:
class Data(object):

    def __init__(self, mtrack_list, data_path, input_patch_size=(26, 26),
                 output_patch_size=(20, 20),
                 batch_size=1024):

        self.mtrack_list = mtrack_list
        self.input_patch_size = input_patch_size
        self.output_patch_size = output_patch_size
        self.batch_size = batch_size

        self.data_path = data_path
        
        (self.train_set,
         self.validation_set,
         self.test_set) = self._train_val_test_split()

        self.train_files = get_file_paths(self.train_set, self.data_path)
        self.validation_files = get_file_paths(
            self.validation_set, self.data_path
        )
        self.test_files = get_file_paths(self.test_set, self.data_path)

    def _train_val_test_split(self):
        full_list = []
        print(len(self.mtrack_list))
        for m in self.mtrack_list:
            globbed = get_file_paths([m], self.data_path)
            if len(globbed) > 0:
                full_list.append(m)

        self.full_list = full_list
        print(len(full_list))
        mtracks = list(mdb.load_multitracks(full_list))
        test_potentials = [
            m.track_id for m in mtracks if m.dataset_version == 'V1'
        ]
        all_others = [
            m.track_id for m in mtracks if m.dataset_version != 'V1'
        ]

        split1 = utils.artist_conditional_split(
            trackid_list=test_potentials, test_size=0.2,
            num_splits=1, random_state=RANDOM_STATE
        )

        test_set = split1[0]['test']
        remaining_tracks = split1[0]['train'] + all_others

        split2 = utils.artist_conditional_split(
            trackid_list=remaining_tracks, test_size=0.15,
            num_splits=1, random_state=RANDOM_STATE
        )

        train_set = split2[0]['train']
        validation_set = split2[0]['test']

        return train_set, validation_set, test_set

    def get_train_generator(self):
        return keras_generator(
            self.train_files,
            input_patch_size=self.input_patch_size,
            output_patch_size=self.output_patch_size,
            batch_size=self.batch_size
        )

    def get_validation_generator(self):
        return keras_generator(
            self.validation_files,
            input_patch_size=self.input_patch_size,
            output_patch_size=self.output_patch_size,
            batch_size=self.batch_size
        )

    def get_test_generator(self):
        return keras_generator(
            self.test_files,
            input_patch_size=self.input_patch_size,
            output_patch_size=self.output_patch_size,
            batch_size=self.batch_size
        )

# load data

In [40]:
data_path = "/scratch/rmb456/multif0_ismir2017/training_data_with_blur/multif0_complete/"
mtrack_list = mdb.TRACK_LIST_V1 + mdb.TRACK_LIST_V2 + mdb.TRACK_LIST_EXTRA
dat = Data(
    mtrack_list, data_path, input_patch_size=(26, 26),
    output_patch_size=(20, 20), batch_size=1024
)

320
188


In [41]:
train_generator = dat.get_train_generator()
validation_generator = dat.get_validation_generator()
test_generator = dat.get_test_generator()

In [44]:
test = next(train_generator.tuples('X', 'Y'))

AttributeError: 'tuple' object has no attribute 'itervalues'

# Build Model

In [34]:
input_shape = (26, 26, 6)
inputs = Input(shape=input_shape)

y1 = Conv2D(
    128, (3, 3), padding='valid', activation='relu', name='bendy'
)(inputs)
y2 = Conv2D(
    128, (3, 3), padding='valid', activation='relu', name='smoothy1'
)(y1)
y3 = Conv2D(
    64, (3, 3), padding='valid', activation='relu', name='smoothy2'
)(y2)
y4 = Conv2D(
    1, (1, 1), padding='valid', activation='sigmoid', name='squishy'
)(y3)
predictions = Reshape((20, 20), name='reshapeeee')(y4)

model = Model(inputs=inputs, outputs=predictions)

# Compile Model

In [35]:
# loss binary cross entropy
model.compile(loss='kld',
              optimizer='sgd')

In [36]:
model.summary(line_length=80)

________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
input_4 (InputLayer)                (None, 26, 26, 6)               0           
________________________________________________________________________________
bendy (Conv2D)                      (None, 24, 24, 128)             7040        
________________________________________________________________________________
smoothy1 (Conv2D)                   (None, 22, 22, 128)             147584      
________________________________________________________________________________
smoothy2 (Conv2D)                   (None, 20, 20, 64)              73792       
________________________________________________________________________________
squishy (Conv2D)                    (None, 20, 20, 1)               65          
________________________________________________________________________________
reshapeeee (Reshape)        

In [27]:
temp_generator = patch_generator(dat.train_files[0], dat.input_patch_size, dat.output_patch_size)

# Fit Model

In [42]:
samples_per_epoch = 1024 * 10
nb_epochs = 10
nb_val_samples = 1024

history = model.fit_generator(
    train_generator.tuples('X', 'Y'), samples_per_epoch, epochs=nb_epochs, verbose=1#,
    #validation_data=validation_generator, validation_steps=nb_val_samples
)

Epoch 1/10


Exception in thread Thread-11:
Traceback (most recent call last):
  File "/home/rmb456/anaconda2/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/rmb456/anaconda2/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/rmb456/anaconda2/lib/python2.7/site-packages/keras/engine/training.py", line 606, in data_generator_task
    generator_output = next(self._generator)
  File "/home/rmb456/anaconda2/lib/python2.7/site-packages/pescador/core.py", line 216, in tuples
    for data in self.generate(**kwargs):
  File "/home/rmb456/anaconda2/lib/python2.7/site-packages/pescador/buffered.py", line 71, in generate
    self.buffer_size)):
  File "/home/rmb456/anaconda2/lib/python2.7/site-packages/pescador/buffered.py", line 99, in buffer_batch
    n += batch_length(x)
  File "/home/rmb456/anaconda2/lib/python2.7/site-packages/pescador/buffered.py", line 199, in batch_length
    for value in six.itervalues(

ValueError: output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`. Found: None

In [ ]:
# plot filters
conv_layer = model.get_layer(name='bendy')
weights = conv_layer.get_weights()
weight_array = weights[0]
plt.figure(figsize=(15, 15))
for i in range(64):
    plt.subplot(8, 8, i+1)
    plt.imshow(weight_array[:, :, 0, i], origin='lower')
    plt.axis('square')

plt.show()

In [ ]:
# plot filters
conv_layer = model.get_layer(name='smoothy')
weights = conv_layer.get_weights()
weight_array = weights[0]
plt.figure(figsize=(15, 15))
for i in range(32):
    plt.subplot(8, 4, i+1)
    plt.imshow(weight_array[:, :, 0, i], origin='lower')
    plt.axis('square')

plt.show()

In [ ]:
def plot_stuff(X, Y, Y_pred):
    sns.set_style('white')
    plt.figure(figsize=(15, 15))
    
    plt.subplot(3, 1, 1)
    plt.title("prediction")
    plt.imshow(Y_pred[0], origin='lower', cmap='hot', vmin=0, vmax=1)
    plt.axis('auto')
#     plt.xlim(a, b)
    plt.colorbar()

    plt.subplot(3, 1, 2)
    plt.title('target')
    plt.imshow(Y[0], origin='lower', cmap='hot', vmin=0, vmax=1)
    plt.axis('auto')
#     plt.xlim(a, b)
    plt.colorbar()

    plt.subplot(3, 1, 3)
    plt.title('input')
    plt.imshow(X[0, :, :, 0], origin='lower', cmap='hot', vmin=0, vmax=1)
    plt.axis('auto')
#     plt.xlim(a, b)
    plt.colorbar()
    plt.show()

In [ ]:
for X, Y in validation_generator:
    Y_pred = model.predict(X)
    plot_stuff(X, Y, Y_pred)
    break
    if np.sum(Y[0].flatten()) > 0:
        plot_stuff(X, Y, Y_pred)
        break

In [ ]:
for X, Y in train_generator:
    for x, y in zip(X, Y):
        if np.sum(y.flatten()) > 0:
            plt.figure(figsize=(15, 15))
            plt.subplot(2, 1, 1)
            plt.imshow(x[:, :, 0], origin='lower', cmap='hot')
            plt.axis('square')

            plt.subplot(2, 1, 2)
            plt.imshow(y, origin='lower', cmap='hot')
            plt.axis('square')
            break

# Evaluate Model

In [ ]:
model.count_params()

In [ ]:
OLAP_KERNEL = np.zeros((20, 20))
for i in range(11):
    for j in range(11):
        val = 0.1*float(i)*0.1*float(j)
        OLAP_KERNEL[i, j] = val
        OLAP_KERNEL[-i, j] = val
        OLAP_KERNEL[i, -j] = val
        OLAP_KERNEL[-i, -j] = val

In [ ]:
def get_full_track_prediction(data_in, model):

    n_harms, n_freqs, n_times = data_in.shape
    n_f, n_t = (20, 20)

    prediction = np.zeros((n_freqs, n_times))

    cqt_patch_generator = stride_cqt(data_in)

    for d in cqt_patch_generator:
        f = d['f']
        t = d['t']
        y_pred = model.predict(d['X'].reshape(1, n_f, n_t, n_harms)).reshape(n_f, n_t)
#         prediction[f: f + n_f, t: t + n_t] += y_pred * OLAP_KERNEL
        prediction[f: f + n_f, t: t + n_t] = y_pred

    return prediction
    

In [ ]:
test_prediction = get_full_track_prediction(dat3['data_in'], model)

In [ ]:
a, b = (10000, 20000)

sns.set_style('white')
plt.figure(figsize=(15, 15))
plt.subplot(3, 1, 1)
plt.imshow(test_prediction**10, origin='lower', cmap='hot')
# plt.imshow((1.0 - test_prediction)**1000, origin='lower', cmap='hot')
plt.axis('auto')
plt.xlim(a, b)
# plt.colorbar()

plt.subplot(3, 1, 2)
plt.imshow(dat3['data_out'], origin='lower', cmap='hot')
plt.axis('auto')
plt.xlim(a, b)

plt.subplot(3, 1, 3)
plt.imshow(dat3['data_in'][0, :, :], origin='lower', cmap='hot')
plt.axis('auto')
plt.xlim(a, b)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.imshow(dat3['data_in'][0], origin='lower', cmap='hot')
plt.axis('auto')
plt.xlim(0, 10000)

In [ ]:
plt.imshow(dat3['data_out'], origin='lower', cmap='hot')
plt.axis('auto')
plt.xlim(0, 10000)

In [ ]:
import numpy as np

In [ ]:
test = np.zeros((100, 100))

In [ ]:
for i in range(50):
    test[i, i] = 1
for j in range(50, 100):
    test[50, j] = 1

In [ ]:
test

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.imshow(test, origin='lower', interpolation='none')
plt.axis('auto')

In [ ]:
from scipy.ndimage import filters

In [ ]:
test_filt = filters.gaussian_filter1d(test, 2, axis=0, mode='constant')
test_filt = test_filt/np.max(test_filt)

In [ ]:
plt.figure(figsize=(15, 8))
plt.imshow(test_filt, origin='lower', interpolation='none')
plt.axis('auto')
plt.colorbar()